# Train Liveness

## Dependencies

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
INIT_LR = 1e-4
BATCH_SIZE = 32
EPOCHS = 50
TARGET_SIZE = (64, 64)

## Data Loader

In [ ]:
train_dir = ""
validation_dir = ""

train_datagen = ImageDataGenerator()

validation_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=BATCH_SIZE,
                                                    target_size=TARGET_SIZE,
                                                    class_mode='binary',
                                                   color_mode="rgb",
                                                   shuffle=True)

validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                              batch_size=BATCH_SIZE,
                                                              target_size=TARGET_SIZE,
                                                              class_mode='binary',
                                                             color_mode="rgb",
                                                             shuffle=False)

## Build Model

In [ ]:
class LivenessNet:
  def __init__(self, width=64, height=64, depth=3, classes=2):
    self.width = width
    self.height = height
    self.depth = depth
    self.classes = classes

  def build(self):
    input_shape = (self.height, self.width, self.depth)

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(16, (3, 3), activation="relu", padding="same", input_shape=input_shape))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(16, (3, 3), activation="relu", padding="same", input_shape=input_shape))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(rate=0.25))

    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same", input_shape=input_shape))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same", input_shape=input_shape))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(rate=0.25))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(64, activation="relu"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(rate=0.5))

    model.add(tf.keras.layers.Dense(self.classes, activation="sigmoid"))

    return model

In [ ]:
model = LivenessNet().build()
model.summary()

## Train Model

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=INIT_LR, decay=INIT_LR/EPOCHS)
step_size_train = train_generator.n // train_generator.batch_size
step_size_validation = validation_generator.samples // validation_generator.batch_size
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=10)

In [ ]:
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator, epochs=EPOCHS, validation_data=validation_generator, verbose=1,
                   steps_per_epoch=step_size_train, validation_steps=step_size_validation,
                   callbacks=[early_stopping])

## Save Model

In [ ]:
model.save("liveness_model.h5")